In [2]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/samples.ipynb")

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Getting the k-mer information for KhoeSan individuals

In [3]:
samples = ['SRR1171049', 'SRR1171058', 'SRR1175182', 'SRR1258307', 
            'SRR1171017', 'SRR1171051', 'SRR1175179', 'SRR1175183', 'SRR1265483',
            'SRR1171020', 'SRR1171052', 'SRR1175180', 'SRR1175184', 'SRR1265484',
            'SRR1171021', 'SRR1171056', 'SRR1175181', 'SRR1258307']